**Лабораторный практикум по курсу «Распознавание диктора», Университет ИТМО, 2021**		

**Лабораторная работа №5. Адаптация и калибровка системы распознавания диктора**

**Цель работы:** изучение методов адаптации и калибровки, используемых на практике при решении задачи голосовой биометрии.

**Краткое описание:** в рамках настоящей лабораторной работы предлагается изучить и реализовать некоторые алгоритмы адаптации и калибровки биометрических систем. В качестве процедур адаптации предлагается рассмотреть адаптацию путём нормализации дикторских эмбеддингов с использованием среднего эмбеддинга, а также адаптацию на основе s-нормализации. Процедуру калибровки предлагается реализовать с помощью метода логистической регрессии. Тестирование системы голосовой биометрии после выполнения процедур адаптации и калибровки предлагается выполнить на основе метрики равновероятной ошибки применительно к задаче верификации.

**Данные:** в качестве данных для выполнения лабораторной работы предлагается использовать тестовую часть базы
[VoxCeleb1](http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1.html) и [VoxCeleb2](http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html)

**Содержание лабораторной работы**

1.	Подготовка данных для выполнения процедуры адаптации.  

2.	Реализация процедуры адаптации с использованием центрирования дикторских эмбеддингов на средний эмбеддинг.  

3.	Реализация процедуры адаптации с использованием s-нормализации.  

4.	Тестирование адаптированной к некоторым условиям биометрической системы.  

5.	Подготовка данных для выполнения процедуры калибровки.  

6.	Реализация простейшего алгоритма для выполнения процедуры калибровки системы голосовой биометрии.  

7.	Тестирование калиброванной системы голосовой биометрии.  

In [ ]:
# IPython extension to reload modules before executing user code
%load_ext autoreload
%autoreload 2

# Import of modules
import os
import sys

sys.path.append(os.path.realpath('..'))

import torch
import numpy as np
import itertools

from math import sqrt, pi
from matplotlib.pyplot import hist, plot, show, grid, title, xlabel, ylabel, legend, axis, imshow
from torch.utils.data import DataLoader
from functools import partial
import glob

import common
from common import download_dataset, extract_dataset, download_protocol, run_voxceleb_convert, part_extract
from common import get_voxceleb_filelist
from common import test_dataset_loader
from common import extract_features, compute_scores_cosine
from common import get_eer
from common import concatenate
from common import tsne
from LoadModel import load_model
from ResNetSE34V2 import MainModel

from exercises import plot_histograms_2sets, mean_embd_norm, s_norm, get_tar_imp_scores

**1. Подготовка тестовых данных и протокола для проведения тестирования системы верификации диктора**

В ходе выполнения лабораторной работы необходимы *тестовые данные*, а также связанный с ними *протокол тестирования*. Данные представляют собой звукозаписи голосов людей мужского и женского пола, сохраненных в формат *wav*. Протокол тестирования организован в виде текстового файла, в каждой строчке которого содержится одна таргет- или импостор-попытка сравнения эталонной и тестовой звукозаписи (рассматривается задача *верификации диктора*, т.е. сравнение «один к одному»). Голоса эталона и теста из *таргет-попытки* (метка «1») соответствуют одному и тому же диктору, а голоса эталона и теста из *импостор-попытки* (метка «0») соответствуют разным дикторам.

Необходимые данные можно скачать по ссылкам, представленным ниже:

1. Выполнить загрузку и распаковку звуковых wav-файлов базы [VoxCeleb1 test set](https://thor.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox1_test_wav.zip) и [VoxCeleb2 test set](https://thor.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox2_test_wav.zip).

2. Преобразовать базу [VoxCeleb2 test set](https://thor.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox2_test_wav.zip) из aac в wav.

3. Выполнить загрузку протокола тестирования [VoxCeleb1-O cleaned](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/veri_test2.txt).

4. Подготовить тестовые и адаптационные *in-domain* данные путем эмулирования телефонного канала. В качестве тестовых рассматриваем данные полученные из базы VoxCeleb1 test set, в качестве адаптационных – из базы VoxCeleb2 test set.

![Рисунок 1](https://analyticsindiamag.com/wp-content/uploads/2020/12/image.png "VoxCeleb. Крупномасштабная аудиовизуальная база данных человеческой речи.")

In [ ]:
# Download VoxCeleb1 (test set)
with open('../data/lists/datasets.txt', 'r') as f:
    lines = f.readlines()

download_dataset(lines, user='voxceleb1902', password='nx0bl2v2', save_path='../data')

In [ ]:
# Download VoxCeleb2 (test set)
with open('../data/lists/datasets2.txt', 'r') as f:
    lines = f.readlines()

download_dataset(lines, user='voxceleb1902', password='nx0bl2v2', save_path='../data')

In [ ]:
# Extract VoxCeleb1 test set and VoxCeleb2 test set
extract_dataset(save_path='../data/voxceleb1_test', fname='../data/vox1_test_wav.zip')
extract_dataset(save_path='../data/voxceleb2_test', fname='../data/vox2_test_aac.zip')

In [ ]:
# If we need rirs for reverberation
with open('../data/lists/augment_datasets.txt', 'r') as f:
    lines = f.readlines()
    
download_dataset(lines, user=None, password=None, save_path='../data')

In [ ]:
part_extract(save_path='../data', fname='../data/rirs_noises.zip', target=['RIRS_NOISES/simulated_rirs/mediumroom', 'RIRS_NOISES/simulated_rirs/smallroom'])

In [ ]:
def aac_to_wav(infile, outfile):
    common.aac_to_wav(infile, outfile)

# Prepare voxceleb2 : convert from aac to wav format
run_voxceleb_convert(input_path='../data/voxceleb2_test/aac',
                     result_path='../data/voxceleb2_test/wav',
                     fun=aac_to_wav)

In [ ]:
def convert_to_in_domain(infile, outfile, rir_list):
    common.reverberate(infile, outfile, rir_list)
#     common.apply_mp3_codec(infile, outfile)

rir_path          = '../data/RIRS_NOISES/simulated_rirs'
rirs = glob.glob(os.path.join(rir_path, '*/*/*.wav'));
    
f = partial(convert_to_in_domain, rir_list=rirs)

run_voxceleb_convert(input_path='../data/voxceleb1_test/wav',
                     result_path='../data/voxceleb1_test/wav_reverb',
                     fun=f,
                     threads=10)
run_voxceleb_convert(input_path='../data/voxceleb2_test/wav',
                     result_path='../data/voxceleb2_test/wav_reverb',
                     fun=f,
                     threads=10)

In [ ]:
# Download VoxCeleb1-O cleaned protocol
with open('../data/lists/protocols.txt', 'r') as f:
    lines = f.readlines()
    
download_protocol(lines, save_path='../data/voxceleb1_test')

**2. Вычисление дикторских моделей для эталонных и тестовых звукозаписей используемого протокола**

Итого, мы имеем 2 тестовые базы:

- *out-of-domain*: исходная тестовая база VoxCeleb1_test в микрофонном канале;
- *in-domain*: целевая тестовая база in-domain в телефонном канале – VoxCeleb1 test set, полученная эмулированием телефонного канала.

Аналогично предыдущей лабораторной работе, построим дикторские модели для наших тестовых баз с использованием предобученной нейронной сети и оценим ее качество на наших in-domain и out-of-domain данных. Для этого необходимо:

1. Описать и инициализировать модель для генерации дикторских эмбеддингов.
2. Вгрузить в инициализированную модель обученные веса.
3. Вычислить эмбеддинги для эталонных и тестовых звукозаписей из исходной базы out-of-domain.
4. Повторить п.3 для второй тестовой базы in-domain.

In [ ]:
# Model initialization
model = MainModel()

In [ ]:
# Load trained model
with open('../data/lists/models.txt', 'r') as f:
    lines = f.readlines()
        
model = load_model(model, lines, save_path='../data/models')

model.eval()

model = model.cuda()

In [ ]:
# Read test protocol
with open('../data/voxceleb1_test/veri_test2.txt', 'r') as f:
    protocol = f.readlines()

# Get a list of unique file names
files = list(itertools.chain(*[x.strip().split()[-2:] for x in protocol]))
setfiles = list(set(files))
setfiles.sort()

Построим эмбеддинги для исходной базы VoxCeleb1 test - out-of-domain тестовой базы.

In [ ]:
# Define test data loader
test_dataset = test_dataset_loader(setfiles, test_path='../data/voxceleb1_test/wav', eval_frames=500, num_eval=1)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8, drop_last=False, sampler=None)

# Extract features for every image
feats_out_of_domain = extract_features(model, test_loader)

Построим эмбеддинги для in-domain тестовой базы (протокол остался прежним, имена файлов тоже, поэтому список setfiles не изменился)

In [ ]:
# Define test data loader
test_dataset = test_dataset_loader(setfiles, test_path='../data/voxceleb1_test/wav_reverb', eval_frames=500, num_eval=1)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8, drop_last=False, sampler=None)

# Extract features for every image
feats_in_domain = extract_features(model, test_loader)

**3. Формирование оценок сравнения эталонных и тестовых дикторских моделей**

Для того, чтобы автоматически принять решении о принадлежности пары сравнения «эталон-тест» к таргет- или импостор-попытке, необходимо сформировать *оценку сравнения* между эталонной и тестовой дикторскими моделями, иначе мы называем их scores (скоры). В простейшем случае подобные сравнения можно выполнить с использованием *евклидовой метрики*, *косинусной метрики* и т.п.

В рамках настоящего пункта необходимо выполнить следующее:

1. Выполнить формирование оценок сравнения эталонных и дикторских моделей с использованием любой подходящей метрики (евклидово расстояние, косинусное расстояние и т.п.).

2. Сформировать списки оценок сравнения, таргет-/импостор-меток и имен эталон-тест в триальных сравнениях.

3. Получить оценки EER и построить графики распределения для тестовых баз in-domain и out-of-domain.

In [ ]:
# Compute scores between enroll and test speaker models
# For in domain and out of domain data
all_scores_in_domain, all_labels_in_domain, all_trials_in_domain = compute_scores_cosine(feats_in_domain, protocol)
all_scores_out_of_domain, all_labels_out_of_domain, all_trials_out_of_domain = compute_scores_cosine(feats_out_of_domain, protocol)


# Compute target and impostor histogram
plot_histograms_2sets(all_scores_in_domain, all_labels_in_domain,
                      all_scores_out_of_domain, all_labels_out_of_domain)

**4. Доменная адаптация**

*Доменная адаптация* рассматривается как проблема настройки параметров модели для случая когда существует большой объём нецелевых данных (out-of-domain data, source data) и малый объём размеченных или неразмеченных целевых данных (in-domain data, target data). В научно-технической литературе по распознаванию диктора часто выделяют следующие адаптационные техники: вычитание среднего, s-норма, CORAL, CORAL+, GRL, MultiReader и т.п. Важно отметить, что доменная адаптация может выполняться на всех этапах конвейера голосовой биометрии, то есть на этапе предобработки, диаризации, построения дикторской модели и т.п.

В рамках настоящей лабораторной работы рассматриваются два классических для области распознавания диктора метода адаптации: вычитание среднего и s-нормализация. 

**4.1. Адаптация на основе вычитания среднего**

Рассмотрим подход адаптации на основе вычитания среднего на уровне эмбеддингов, сгенерированных с использованием нейросетевой модели, обученной на нецелевых данных. Основные этапы данного подхода описаны ниже:

1. Рассмотрим верификационный протокол тестирования, в котором эталонные и тестовые звукозаписи соответствуют одним и тем же целевым условиям.
2. Вычислим эталонные и тестовые дикторские эмбеддинги с использованием имеющейся нейросетевой модели.
3. Подготовим адаптационное множество данных, соответствующее целевым условиям.
4. Вычислим дикторские эмбеддинги, имеющейся нейросетевой моделью, для данных из адаптационного множества и рассчитаем для него средний дикторский эмбеддинг.
5. Центрируем эталонные и тестовые дикторские эмбеддинги на средний дикторский эмбеддинг адаптационного множества.
6. Подготовим модель сравнения дикторских эмбеддингов и выполним сравнение центрированных эталонных и тестовых дикторских эмбеддингов.

Собственно, этапы, описанные выше, и нужно реализовать в рамках настоящего пункта. Ниже предполагается, что адаптационное множество построено на основе базы VoxCeleb2 test set, а тестирование будет выполнено на основе базы VoxCeleb1 test set по отношению к протоколу VoxCeleb1-O cleaned. При при проведении тестов в одном случае база VoxCeleb1 test set будет рассматривается в оригинальном виде, а в другом – в искажённом, в котором предполагается, что выбранное искажение моделирует целевые условия. Необходимо отметить, что поскольку, изначально, данные из базы VoxCeleb2 test set соответствуют условиям, в которых записана база VoxCeleb1 test set, то для использования базы VoxCeleb2 test set, в задаче адаптации применительно к верификационному тесту на искаженных данных базы VoxCeleb1 test set, необходимо подвергнуть её искажениям, соответствующим целевым условиям. Подобная процедура выполнена выше в обном из пунктов лабораторной работы. Итогом выполнения настоящего пункта является оценка качества полученного после адаптации к целевым условиям верификационного решения. Качество предлагается проанализировать с использованием метрики EER и гистограмм распределения таргет- и импостор-оценок.

In [ ]:
setfiles = get_voxceleb_filelist(input_path='../data/voxceleb2_test/wav_reverb')
setfiles.sort()

# Define test data loader
adapt_dataset = test_dataset_loader(setfiles, test_path='../data/voxceleb2_test/wav_reverb', eval_frames=1000, num_eval=1)
adapt_loader  = torch.utils.data.DataLoader(adapt_dataset, batch_size=1, shuffle=False, num_workers=8, drop_last=False, sampler=None)

# Extract features for every image
feats_adapt = extract_features(model, adapt_loader)

In [ ]:
tsne_dict = {'in_domain': feats_in_domain,
             'out_of_domain': feats_out_of_domain,
                          'adapt': feats_adapt,
             #              'in_domain_adapt': feats_in_domain_adapted,
             #              'out_domain_adapt': feats_out_of_domain_adapted
             }
tsne(tsne_dict)

In [ ]:
feats_in_domain_adapted = mean_embd_norm(feats_in_domain, feats_adapt)
# if we adapt out of domain data we will have worse results:
# feats_out_of_domain_adapted = mean_embd_norm(feats_out_of_domain, feats_adapt)
feats_adapt_centered = mean_embd_norm(feats_adapt, feats_adapt)

# Compute scores between enroll and test speaker models
# For in domain and out of domain data
all_scores_in_domain, all_labels_in_domain, all_trials_in_domain = compute_scores_cosine(feats_in_domain_adapted, protocol)
all_scores_out_of_domain, all_labels_out_of_domain, all_trials_out_of_domain = compute_scores_cosine(feats_out_of_domain, protocol)


# Compute target and impostor histogram
plot_histograms_2sets(all_scores_in_domain, all_labels_in_domain,
                      all_scores_out_of_domain, all_labels_out_of_domain)

**4.2. Адаптация на основе s-нормы**

Адаптация с помощью s-нормы предполагает выполнение процедуры нормализации оценок сравнения эталонов и тестов с целью уменьшения вариативности в попытках сравнения, которое ведёт к более лучшей калибровке и более надёжному выбору порога принятия решения. При выполнении адаптации на основе s-нормы необходимо выполнить следующую последовательность действий:

1. Выполнить сравнение эталонной и тестовой дикторских моделей в рамках одной попытки, получив оценку.
2. Ввести в рассмотрение импосторную когорту, состоящую из дикторских моделей, которые не пересекаются, в смысле дикторских меток, с эталонной и тестовой моделями в рамках одной попытки.
3. Сравнить эталонную дикторскую модель с каждой моделью из импосторной когорты, получив оценки сравнения; упорядочить оценки сравнения по убыванию схожести моделей; выбрать первые $N$ оценок (самые сложные оценки сравнения) и вычислить для них среднее, $\mu_e$, и среднеквадратическое отклонение, $\sigma_e$.
4. Повторить процедуру выше для тестовой дикторской модели, получив значения $\mu_t$ и $\sigma_t$.
5. Выполнить нормализацию оценки сравнения $S$ для рассматриваемой попытки: $\hat{S}=\frac{S - \mu_e}{2\sigma_e} + \frac{S - \mu_t}{2\sigma_t}$.

В рамках настоящего пункта необходимо реализовать этапы, описанные выше. Как и ранее, предполагается, что данные из базы VoxCeleb2 test set будут использоваться в качестве адаптационного множества. Поскольку основу адаптации с помощью s-нормы составляет импосторная когорта, то искажённые под целевые условия данные из базы VoxCeleb2 test set в рамках настоящего пункта предполагается использовать для построения этой когорты. При при проведении тестов в одном случае база VoxCeleb1 test set будет рассматривается в оригинальном виде, а в другом – в искажённом, в котором предполагается, что выбранное искажение моделирует целевые условия. Итогом выполнения настоящего пункта является оценка качества полученного после адаптации к целевым условиям верификационного решения. Качество предлагается проанализировать с использованием метрики EER и гистограмм распределения таргет- и импостор-оценок.

In [ ]:
all_scores_in_domain, all_labels_in_domain, all_trials_in_domain = s_norm(feats_in_domain, protocol, feats_adapt, N_s=100, eps=0.6)
all_scores_out_of_domain, all_labels_out_of_domain, all_trials_out_of_domain = s_norm(feats_out_of_domain, protocol, feats_adapt, N_s=100, eps=0.6)
# Compute target and impostor histogram
plot_histograms_2sets(all_scores_in_domain, all_labels_in_domain,
                      all_scores_out_of_domain, all_labels_out_of_domain)

**5. Калибровка системы голосовой биометрии**

Результатом процедуры сравнения дикторских моделей являются «сырые» оценки сравнения. Основными проблемами при использовании «сырых» оценок являются:

1. Порог принятия решения для выбранной рабочей точки полностью привязан к системе распознавания диктора (если существует $N$ систем, то существует $N$ порогов).
2. Для произвольной выходной оценки сравнения нет информации о вероятности принадлежности к статистическим гипотезам.
3. Сложно учесть влияние параметров качества речевого сигнала на выходные оценки.

*Калибровка* является решением указанных проблем. Необходимость калибровки в сообществе по распознаванию диктора обусловлена следующим:

1. Представление оценок сравнения в калиброванном виде для конкурсов, например, NIST SRE. Качество калибровки учитывается наравне с качеством верификации.
2. В коммерческих системах распознавания по голосу, так как на практике для заданной оценки сравнения важно знать вероятность статистической гипотезы.

В простейшем случае модель калибровки для системы распознавания диктора может быть описаны в виде линейной функциональной зависимости: $S_c = aS + b$. Здесь $a$ и $b$ – это параметры линейной модели калибровки, $S$ – это «сырое» значение оценки сравнения, а $S_c$ – значение оценки сравнения после калибровки.

Для обучения параметров $a$ и $b$ линейной модели калибровки можно воспользоваться стоимостной функцией для *логистической регресии*. Можно доказать, что при решении задачи верификации апостериорная вероятность принадлежности пары сравнения «эталон-тест» к таргет-паре (нулевая гипотеза, $H_0$) для заданного значения калиброванной оценки сравнения эталона и теста может быть выражена следующим образом: $P(H_0|S_c) = \frac{1}{1 + e^{-S_c + T_{act}^{LLR}}}$. Здесь $T_{act}^{LLR}$ – это величина оптимального порога принятия решения в пространстве логарифма отношения правдоподобия. Фактически, величина $T_{act}^{LLR}$ задаёт желаемую теоретически рабочую точку биометрической системы. Обычно в научно-технической литературе по распознаванию диктора величина $T_{act}^{LLR}$ задается в следующем виде: $T_{act}^{LLR} = ln\left ( \frac{1 - P(H_0)}{P(H_0)} \right )$. Здесь $P(H_0)$ – это априорная вероятность таргет-гипотезы. Аналогичным образом, что и выше, можно записать апостериорную вероятность принадлежности пары сравнения «эталон-тест» к импостор-паре (альтернативная гипотеза, $H_1$) для заданного значения калиброванной оценки сравнения эталона и теста: $P(H_1|S_c) = \frac{1}{1 + e^{S_c - T_{act}^{LLR}}}$. Предполагая, что априорные вероятность выпадения таргет- и импостор-попыток равны $P(H_0)$ и $P(H_1)$, соответственно, можно записать следующее выражение для стоимостной функции, которая позволит путём численной оптимизации, вычислить оптимальные значения параметров $a$ и $b$ через минимизацию стоимостной функции:

$$J(a,b) = \frac{P(H_0)}{N_0}\sum_{i=1}^{N_0}ln\left ( 1 + e^{-(aS_i + b) + T_{act}^{LLR}} \right ) + \frac{1 - P(H_0)}{N_1}\sum_{i=1}^{N_1}ln\left ( 1 + e^{aS_i + b - T_{act}^{LLR}} \right ),$$

где $N_0$ и $N_1$ – это количество таргет- и импостор-пар в калибровочном протоколе тестирования для верификации, который необходимо подготовить отдельно перед обучением калибровочной модели. Попытки сравнения «эталон-тест» в калибровочном протоколе должны быть схожими с теми, что будут встречаться при практическом использовании итоговой системы распознавания диктора, в смысле длительностей эталонной и тестовой звукозаписей, условий записи фонограмм и т.п.

Для оценки качества модели калибровки системы голосовой биометрии можно воспользоваться подходом представленным ниже.

Пусть в настоящей лабораторной работе *байесовский риск* задан в следующем виде: $\overline{C} = P(D_1|H_0)P(H_0) + P(D_0|H_1)P(H_1)/P(H_0)$. Здесь $P(D_1|H_0)$ – это вероятность ошибки первого рода, а $P(D_0|H_1)$ – вероятность ошибки второго рода. Биометрическая система считается откалиброванной, если практический минимум байесовского риска, $\overline{C}_{min}$, вычисленный в эмпирически подобранном пороге $T_{min}^{LLR}$ по отношению к некоторому протоколу тестирования для задачи верификации, совпадает с теоретическим минимум байесовского риска, $\overline{C}_{act}$, вычисленным в точке $T_{act}^{LLR}$.

В рамках настоящего пункта необходимо обучить линейную модель калибровки. Для решения указанной задачи необходимо выполнить следующие действия:

1. Подготовить калибровочный протокол для обучения модели калибровки.
2. Задать рабочую точку, которая определит величину $T_{act}^{LLR}$.
3. Обучить калибровочную модель путём минимизации стоимостной функции для логистической регрессии с использованием методов численной оптимизации первого порядка.
4. Посчитать значения $\overline{C}_{min}$ и $\overline{C}_{act}$, а также $T_{min}^{LLR}$ и $T_{act}^{LLR}$ для выбранных протоколов тестирования до выполнения и после выполнения процедуры калибровки.

In [ ]:
# Download meta data for adapt_set datasets
with open('../data/lists/protocols2.txt', 'r') as f:
    lines = f.readlines()

download_protocol(lines, save_path='../data/voxceleb2_test')

In [ ]:
# TODO: finish it
import CalibrationModel
from CalibrationDataset import CalibrationDataset

tar, imp = get_tar_imp_scores(all_scores_in_domain, all_labels_in_domain)
test_set = CalibrationDataset(target_scores=tar, impostor_scores=imp)

# generate adapt protocol
meta_vox2 = pd.read_csv('../data/voxceleb2_test/vox2_meta.csv', sep=' ,')
meta_vox2 = meta_vox2.rename(columns={"VoxCeleb2 ID": 'id', "Gender": "gender"})

generate_protocol(adapt_dataset.test_list, adapt_dataset.test_list, meta_enroll=meta_vox2, meta_test=meta_vox2)

tar, imp = get_tar_imp_scores(all_scores_in_domain, all_labels_in_domain)
train_set = CalibrationDataset(target_scores=tar, impostor_scores=imp)

calibration_model = CalibrationModel.LinearModel()
train_calibration(calibration_model, train_set, test_set, system_name, num_epochs=10)

**7. Контрольные вопросы**
											
1. Что такое доменная адаптация?
2. Как выполнить процедуру адаптации через вычисление среднего дикторского эмбеддинга адаптационного множества?
3. Как выполнить процедуру адаптации на основе s-нормализации?
4. Для какой цели решается задача калибровки?
5. Что такое «хорошая» калибровка?
6. Какие модели калибровки систем голосовой биометрии рассматриваются на практике?
7. Как обучить калибровочную модель с использованием метода логистической регресии?

**8. Список литературы**

1. Matějka P., Novotný O., Plchot O., Burget L., Sánchez M.D., Černocký J. Analysis of score normalization in multilingual speaker recognition // Proc. Interspeech 2017, 2017. P. 1567–1571 ([ссылка](https://www.isca-speech.org/archive/pdfs/interspeech_2017/matejka17_interspeech.pdf)).

2. Gusev A., Volokhov V., Vinogradova A., Andzhukaev T., Shulipa A., Novoselov S., Pekhovsky T., Kozlov A. STC-Innovation speaker recognition systems for far-field speaker verification challenge 2020 // Proc. Interspeech 2020, 2020. P. 3466–3470 ([ссылка](https://www.isca-speech.org/archive/pdfs/interspeech_2020/gusev20_interspeech.pdf)).

3. Brümmer N., Villiers E. The BOSARIS toolkit: theory, algorithms and code for surviving the new DCF // arXiv:1304.2865 [stat.AP] ([ссылка](https://arxiv.org/pdf/1304.2865.pdf)).